In [ ]:
import pymupdf
import fitz
import pytesseract
from PIL import Image
import os

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

input_dir = "input"  
output_dir = "output_texts"

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.pdf'):
        input_pdf_path = os.path.join(input_dir, filename)

        try:
            doc = fitz.open(input_pdf_path)
        except Exception as e:
            print(f"Error opening PDF file {input_pdf_path}: {e}")
            quit()

        extracted_text = ""

        for page_number in range(len(doc)):
            try:
                page = doc.load_page(page_number)

                pix = page.get_pixmap()

                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

                text = pytesseract.image_to_string(img)

                extracted_text += f"=== Page {page_number + 1} ===\n{text}\n\n"
            except Exception as e:
                print(f"Error processing page {page_number + 1} of {input_pdf_path}: {e}")
                continue

        doc.close()

        print(extracted_text)

        output_text_path = os.path.join(output_dir, f"{filename.replace('.pdf', '_extracted.txt')}")
        with open(output_text_path, "w", encoding="utf-8") as output_file:
            output_file.write(extracted_text)

        print(f"Text extracted and saved to {output_text_path}")

print("Processing complete.")

In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import os
import re
import pandas as pd

# Set the path to the Tesseract executable (change this according to your installation path)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Directory containing the input PDF files
input_dir = "input"  # Replace with your input directory path
output_dir = "output_texts"
output_excel_path = "resumes_data.xlsx"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Initialize a list to store resume data
resumes_data = []

# Regex patterns for extracting specific details
name_pattern = re.compile(r'\bName\s*:\s*(.*)', re.IGNORECASE)
email_pattern = re.compile(r'\bEmail\s*:\s*([\w\.-]+@[\w\.-]+\.\w+)', re.IGNORECASE)
phone_pattern = re.compile(r'\bPhone\s*:\s*(\+?\d[\d\s-]{8,}\d)', re.IGNORECASE)
# Add more patterns as needed for other details

# Iterate over all PDF files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.pdf'):
        input_pdf_path = os.path.join(input_dir, filename)

        try:
            # Open the PDF file
            doc = fitz.open(input_pdf_path)
        except Exception as e:
            print(f"Error opening PDF file {input_pdf_path}: {e}")
            continue

        # Initialize empty string to store extracted text
        extracted_text = ""

        # Iterate through each page
        for page_number in range(len(doc)):
            try:
                # Load the current page
                page = doc.load_page(page_number)
                pix = page.get_pixmap()
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

                # Perform OCR on the image using pytesseract
                text = pytesseract.image_to_string(img)

                # Append extracted text to the output string
                extracted_text += f"=== Page {page_number + 1} ===\n{text}\n\n"
            except Exception as e:
                print(f"Error processing page {page_number + 1} of {input_pdf_path}: {e}")
                continue

        # Close the PDF file
        doc.close()

        # Extract specific details using regex
        name = re.search(name_pattern, extracted_text)
        email = re.search(email_pattern, extracted_text)
        phone = re.search(phone_pattern, extracted_text)

        # Get the matched groups or set default values
        name = name.group(1).strip() if name else "Not Found"
        email = email.group(1).strip() if email else "Not Found"
        phone = phone.group(1).strip() if phone else "Not Found"

        # Append the extracted details to the list
        resumes_data.append({
            "Filename": filename,
            "Name": name,
            "Email": email,
            "Phone": phone,
            # Add more fields as needed
        })

        # Print or save the extracted text
        print(extracted_text)

        # Write the extracted text to a file (for debugging purposes)
        output_text_path = os.path.join(output_dir, f"{filename.replace('.pdf', '_extracted.txt')}")
        with open(output_text_path, "w", encoding="utf-8") as output_file:
            output_file.write(extracted_text)

        print(f"Text extracted and saved to {output_text_path}")

# Create a DataFrame from the extracted resume data
df = pd.DataFrame(resumes_data)

# Save the DataFrame to an Excel file
df.to_excel(output_excel_path, index=False)

print("Processing complete and data saved to Excel.")


In [ ]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import os
import re
import pandas as pd

# Set the path to the Tesseract executable (change this according to your installation path)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Directory containing the input PDF files
input_dir = "input"  # Replace with your input directory path
output_dir = "output_texts"
output_excel_path = "resumes_data.xlsx"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Regex patterns for extracting specific details
name_pattern = re.compile(r'\bName\s*:\s*(.*)', re.IGNORECASE)
email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', re.IGNORECASE)
phone_pattern = re.compile(r'\bPhone\s*:\s*(\+?\d[\d\s-]{8,}\d)', re.IGNORECASE)
# Add more patterns as needed for other details

# Initialize a list to store resume data
resumes_data = []

# Iterate over all PDF files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.pdf'):
        input_pdf_path = os.path.join(input_dir, filename)

        try:
            # Open the PDF file
            doc = fitz.open(input_pdf_path)
        except Exception as e:
            print(f"Error opening PDF file {input_pdf_path}: {e}")
            continue

        # Initialize empty string to store extracted text
        extracted_text = ""

        # Iterate through each page
        for page_number in range(len(doc)):
            try:
                # Load the current page
                page = doc.load_page(page_number)
                pix = page.get_pixmap()
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

                # Perform OCR on the image using pytesseract
                text = pytesseract.image_to_string(img)

                # Append extracted text to the output string
                extracted_text += f"=== Page {page_number + 1} ===\n{text}\n\n"
            except Exception as e:
                print(f"Error processing page {page_number + 1} of {input_pdf_path}: {e}")
                continue

        # Close the PDF file
        doc.close()

        # Extract specific details using regex
        name_match = re.search(name_pattern, extracted_text)
        email_match = re.search(email_pattern, extracted_text)
        phone_match = re.search(phone_pattern, extracted_text)

        # Get the matched groups or set default values
        name = name_match.group(1).strip() if name_match else "Not Found"
        email = email_match.group(0).strip() if email_match else "Not Found"
        phone = phone_match.group(1).strip() if phone_match else "Not Found"

        # Append the extracted details to the list
        resumes_data.append({
            "Filename": filename,
            "Name": name,
            "Email": email,
            "Phone": phone,
            # Add more fields as needed
        })

        # Print or save the extracted text
        print(extracted_text)

        # Write the extracted text to a file (for debugging purposes)
        output_text_path = os.path.join(output_dir, f"{filename.replace('.pdf', '_extracted.txt')}")
        with open(output_text_path, "w", encoding="utf-8") as output_file:
            output_file.write(extracted_text)

        print(f"Text extracted and saved to {output_text_path}")

# Create a DataFrame from the extracted resume data
df = pd.DataFrame(resumes_data)

# Save the DataFrame to an Excel file
df.to_excel(output_excel_path, index=False)

print("Processing complete and data saved to Excel.")
